# Accumulation Score

Analysis of abundance or depletion of a given interaction compared to random. One partner of a given interaction is kept constant.

### Example:   
Study of two celltypes (e.g. astrocytes and neurons). Number of neurons is counted in the neighborhood with a radius of maxDistNeighbor of each astrocyte. The labels are then shuffled n_combinations times and the number of neuron in astrocyte neighborhood is counted for each shuffle. The count of the unshuffled data is expressed as a z-score of the shuffled count distribtuion. The labels of cells classified as astrocyte are not shuffled.

### Example data set:  
Multi-channel image data of one core was analyzed in QuPath, as demonstrated in https://www.youtube.com/watch?v=WTAgXpuuqNY.
Detection Measurements were exported as a .txt-file. The exported results file list the cells in one core, characterized by their position in X- and Y (in micrometer) and a cell class.

The code can be re-used according to license: BSD-3-Clause.  
October 2022, Anna Klemm, BioImage Informatics Facility, SciLifeLab/NBIS, biif@scilifelab.se




In [1]:
import numpy as np
import pandas as pd
import scipy.spatial as spatial

In [16]:
# paths to data sets
path_data = r'QuPath_output.txt'
path_output_directory = ""

## import and inspect data


In [3]:
data_all = pd.read_csv(path_data, delimiter = '\t')
data_all.head()


,Image,Name,Class,Parent,ROI,Centroid X µm,Centroid Y µm,Nucleus: Area,Nucleus: Perimeter,Nucleus: Circularity,...,Cytoplasm: CD34 min,Cytoplasm: GFAP mean,Cytoplasm: GFAP std dev,Cytoplasm: GFAP max,Cytoplasm: GFAP min,Cytoplasm: Autofluorescence mean,Cytoplasm: Autofluorescence std dev,Cytoplasm: Autofluorescence max,Cytoplasm: Autofluorescence min,Nucleus/Cell area ratio
0,"LGG TMA 5_2_Core[1,10,B]_[6246,50326]_componen...",IBA1,IBA1,PathAnnotationObject,Polygon,916.38,24.70,29.75,27.0900,0.5094,...,0.0,0.6368,0.1347,1.0032,0.3157,2.6541,0.5555,4.3712,1.5759,0.2966
1,"LGG TMA 5_2_Core[1,10,B]_[6246,50326]_componen...",IBA1,IBA1,PathAnnotationObject,Polygon,932.79,24.83,23.25,19.1744,0.7947,...,0.0,0.1667,0.1167,0.7293,0.0370,5.3507,2.6677,14.4936,0.9215,0.2968
2,"LGG TMA 5_2_Core[1,10,B]_[6246,50326]_componen...",IBA1,IBA1,PathAnnotationObject,Polygon,901.54,25.70,29.75,19.6645,0.9668,...,0.0,0.4672,0.1844,1.0391,0.0989,3.5550,1.7806,10.5589,0.7595,0.2672
3,"LGG TMA 5_2_Core[1,10,B]_[6246,50326]_componen...",IBA1,IBA1,PathAnnotationObject,Polygon,1022.30,28.78,54.75,37.6975,0.4841,...,0.0,0.6313,0.5099,1.8562,0.0172,2.9378,1.8803,9.3204,0.1590,0.2217
4,"LGG TMA 5_2_Core[1,10,B]_[6246,50326]_componen...",NeuroC: IBA1,NeuroC: IBA1,PathAnnotationObject,Polygon,984.46,29.32,56.50,30.4461,0.7659,...,0.0,0.6036,0.5751,2.0294,0.0180,3.3174,2.7253,9.6993,0.0147,0.2365


## Parameter settings

In [11]:
# input data parameters
class_column = 'Class'
x_label = 'Centroid X µm'
y_label = 'Centroid Y µm'

# parameter for neighborhood analysis
maxDistNeighbor = 50 # distance for neighborhood analysis, dimensions as in input data, e.g. micrometer
nSimulations = 100 # number of shuffled random data sets

## Data clean-up and inspection of available classes

In [12]:
# Quality control: get number of unclassified cells
data_all['Class'].fillna('unclassified', inplace = True)
data_all['Class'].value_counts()

unclassified             2179
IBA1                     1050
NeuroC: mutIHD1           929
NeuroC                    761
mutIHD1                   279
NeuroC: IBA1              114
IBA1: mutIHD1              32
NeuroC: IBA1: mutIHD1      18
NeuroC: Ki67               12
Ki67                       10
NeuroC: mutIHD1: Ki67       5
mutIHD1: Ki67               1
Name: Class, dtype: int64

## Calculation of the accumulation score

In [13]:
# get list of classes
list_partners = data_all['Class'].unique()

# calculating the accumulation scores in different images
samples = data_all['Image'].unique()

n_combinations = np.shape(list_partners)[0] * (np.shape(list_partners)[0] - 1)
allCores_accScores_vector = np.empty((0, n_combinations), float)
nSamples = samples.size


list_p2 = list_partners

for i in range(nSamples):
    print(samples[i])
    # data that contains x, y, and label (= cell type, gene)
    # expected data-type is a pandas dataframe with columns for the x-coord, y-coord, and a label for the cell/gene/etc identity
    data_df = data_all[data_all['Image'] == samples[i]]


    data_df = data_df.reset_index() 
    data_df = data_df.rename(columns={class_column : 'Label'})

    labels_pairs = [] #collecting labels of the different pair combinations

    accScore_all = []
    count_p1_p2_all = []

    for pair_partner1 in list_partners:
        ## Analyze spatial connection using cKDTree
        cenX_list = list(data_df[x_label])
        cenY_list = list(data_df[y_label]) 
        points = np.column_stack((cenX_list, cenY_list))
        D = maxDistNeighbor
        point_tree = spatial.cKDTree(points)

        # collect all neighbors in distance maxDistNeighbor around pair_partner1 data points
        p1_cells = list(data_df[data_df.Label == pair_partner1].index)
        labels = data_df.Label.to_numpy()

        original_classes = np.array(data_df.Label)
        p1_neighbors = []
        # group of neighborhood objects always contains the seed
        for i, group  in enumerate(point_tree.query_ball_point(points[p1_cells], D)):
            p1_neighbors.extend(group)


        # create random draws
        n_random_p2_collected = dict.fromkeys(list_p2)

        # get labels of non-P1 data points
        ds_nonP1_labels = data_df.Label[data_df.Label != pair_partner1].copy()
        data_df_withRandomLabels = data_df.copy() #first a copy, next shuffling

        for draw in range(nSimulations):
        # shuffle labels
            random_labels = ds_nonP1_labels.sample(frac=1)
            ind = data_df.Label[data_df.Label != pair_partner1].copy().index
            random_label_column_name = "random" + str(draw)
            random_labels.index = ind
            data_df_withRandomLabels[random_label_column_name] = random_labels
            

            # put back labels of p1 data points
            P1_labels = data_df.Label[data_df.Label == pair_partner1].copy()
            # collect random_labels for visualization
            random_labels_all = pd.concat([random_labels, P1_labels])

            # get number of p2 cells in neighborhoods
            for pair_partner2 in list_p2:
                n_random_p2 = sum(random_labels_all[p1_neighbors] == pair_partner2)
                
                if (draw == 0):
                    n_random_p2_collected[pair_partner2] = [n_random_p2]
                
                else:
                    n_random_p2_collected[pair_partner2].append(n_random_p2)


        n_random_p2_collected
        
        # calculate accumulation Score and compare with original value

        def getAccumulationScore(collect_rnd_nPairs, count_p1_p2, pair_partner1, pair_partner2):
            random_mean = np.mean(collect_rnd_nPairs)
            random_std = np.std(collect_rnd_nPairs)
            if random_std != 0:
                accScore = (count_p1_p2 - random_mean) / random_std
            else:
                accScore = np.nan
            return accScore


        accScore_collected = []

        for pair_partner2 in list_p2:
            if pair_partner2 != pair_partner1: #without p1-p1 
                
                labels_pairs.append(pair_partner1 + '_' + pair_partner2)
                
                count_p1_p2 = sum(labels[p1_neighbors] == pair_partner2)
                counts_random = n_random_p2_collected[pair_partner2]
                accScore = getAccumulationScore(counts_random, count_p1_p2, pair_partner1, pair_partner2)
                accScore_collected.append(accScore)
                count_p1_p2_all.append(count_p1_p2)
        accScore_all.append(accScore_collected)        
    accScores_np = np.array(accScore_all)
    class_counts = data_df['Label'].value_counts()


    accScores_vector = np.reshape(accScores_np, (1, accScores_np.size))
    allCores_accScores_vector = np.append(allCores_accScores_vector, accScores_vector, axis=0)


LGG TMA 5_2_Core[1,10,B]_[6246,50326]_component_data.tif - resolution #1


In [15]:
# create data frame for output as excel
allCores_accScores_df = pd.DataFrame(allCores_accScores_vector, index = samples, columns= labels_pairs )
allCores_accScores_df.to_excel(path_output_directory + 'accScores.xlsx')
